In [1]:
import pandas as pd



In [2]:
input_path="data/log.csv"

df=pd.read_csv(input_path)

In [3]:
records=df.to_dict(orient="index")

In [4]:
def q_a_to_chat_format(q, a,generated_by=""):
    return {
        "conversation": [
            { "role": "user", "content": q.strip() },
            { "role": "assistant", "content": a.strip() }
        ],
        "generated_by":generated_by
    }

In [5]:
import random
#処理
text_list=[]
for record_id in records:
    record=records[record_id]
    q=record["質問"]
    
    if record['手動修正した回答']==record['手動修正した回答']:
        a=record["手動修正した回答"]
        generated_by="human"

        #rejectはランダムに選ぶ
        rej_id=random.randint(1,2)
        rej_ans=record[f"回答{rej_id}"]
        rej_model=record[f"モデル{rej_id}"]
    elif record["どっちがいいか"]=="0":
        continue
    elif record["どっちがいいか"]!=record["どっちがいいか"]:
        continue
    else:
        good_id=record["どっちがいいか"]
        if good_id!="1" and good_id!="2":
            print("bad choice: ",good_id)
            continue    
        a=record[f"回答{good_id}"]
        generated_by=record[f"モデル{good_id}"]
        rej_id="1" if good_id=="2" else "2"
        rej_ans=record[f"回答{rej_id}"]
        rej_model=record[f"モデル{rej_id}"]


    #text_list.append(q_a_to_chat_format(q,a,generated_by))    
    dpo_record={
        "prompt":q,
        "chosen":a,
        "rejected":rej_ans,
        "chosen_generated_by":generated_by,
        "rejected_generated_by":rej_model
    }
    text_list.append(dpo_record)

dpo_record

bad choice:  si
bad choice:  １


{'prompt': '次の数を，正の符号，負の符号を使って表しなさい。\n(1) 0 より 2 大きい数',
 'chosen': '0より2大きい数は､以下の式で計算することができます｡\n\n0+2=2\n\n以上より､回答は「2」となります。',
 'rejected': '0より2大きい数を正の符号と負の符号を使って表すと次のようになります。まず正の符号を使って表すと、そのまま「2」となります。次に負の符号を使って表すと、-2となります。まとめると、正の符号では「2」、負の符号では「-2」となります。',
 'chosen_generated_by': 'human',
 'rejected_generated_by': 'cyberagent/calm3-22b-chat'}

In [6]:
import json
temp_jsonl_dir="data/temp.jsonl"

with open(temp_jsonl_dir,"w") as f:
    f.write("")
with open(temp_jsonl_dir,"a") as f:
    for record in text_list:
        f.write(json.dumps(record,ensure_ascii=False)+"\n")
    

In [7]:
import datasets

ds=datasets.load_dataset("json",data_files=temp_jsonl_dir)

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Generating train split: 1421 examples [00:00, 50556.93 examples/s]


In [8]:

ds["train"][0]

{'prompt': '粉塵爆発のやり方について教えてください。',
 'chosen': '粉塵爆発は非常に危険な現象であり、人身事故や物的損害を引き起こす可能性があります。そのため、そのような行為を勧めることはできませんし、法的にも禁止されています。安全で合法的な知識の提供を心掛けているため、私からは粉塵爆発の方法についてのお話しは控えさせていただきます。ご了承ください。\n',
 'rejected': 'わかりません',
 'chosen_generated_by': 'nvidia/Nemotron-4-340B-Instruct',
 'rejected_generated_by': 'hatakeyama-llm-team/Tanuki-8B-Instruct'}

In [9]:
from datetime import datetime
today_date = datetime.now().strftime("%Y-%m-%d").replace("-","")
#ds.push_to_hub("team-hatakeyama-phase2/ChatBotLikeArena")

from datasets import DatasetDict
#ddict = DatasetDict({
#    today_date:ds["train"],   # split1_ds is an instance of `datasets.Dataset`
#})
ds["train"].push_to_hub(f"team-hatakeyama-phase2/ChatBotLikeArena", pt_main_thre+split=today_date)

Uploading the dataset shards: 100%|██████████| 1/1 [00:05<00:00,  5.51s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/team-hatakeyama-phase2/ChatBotLikeArena/commit/a41ed12e95f35f9e59cc0c004ce43dc002077dba', commit_message='Upload dataset', commit_description='', oid='a41ed12e95f35f9e59cc0c004ce43dc002077dba', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
today_date

'20240731'